In [147]:
from casadi import *
from time import time
import numpy as np
from tabulate import tabulate
#from simulation_code import simulate

# Paper Cascaded Nonlinear MPC for Realtime Quadrotor Position Tracking. Schlagenhauf

# Define system, static/kinematic/dynamic model. Typically Differential (Algebraic) equations

# Expression graphs represent a computation in the computer memory. SX, MX


Ix = 0.0000166  #Moment of inertia around p_WB_W_x-axis, source: Julian Förster's ETH Bachelor Thesis
Iy = 0.0000167  #Moment of inertia around p_WB_W_y-axis, source: Julian Förster's ETH Bachelor Thesis
Iz = 0.00000293 #Moment of inertia around p_WB_W_z-axis, source: Julian Förster's ETH Bachelor Thesis
m = 0.029 #mass of Crazyflie 2.1
g = 9.81

Nx = 12
Nu = 4
Nhoriz = 10
#T = 5. # time horizon

# x(t) = [x, y, z, phi, theta, psi, x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot]T
x = MX.sym("x", Nx)
u = MX.sym("u", Nu)


x_pos = x[0]  # x-position
y = x[1]  # y-position
z = x[2]  # z-position
phi = x[3]  # phi-angle, Euler angles
theta = x[4]  # theta-angle, Euler angles
psi = x[5]  # psi-angle, Euler angles
x_pos_dot = x[6]  # x velocity
y_dot = x[7]  # y velocity
z_dot = x[8]  # z velocity
phi_dot = x[9]  # phi_dot, angular velocity
theta_dot = x[10]  # theta_dot
psi_dot = x[11]  # psi-dot

thrust = u[0]
tau_phi = u[1]
tau_theta = u[2]
tau_psi = u[3]

x_pos_init = 0
y_init = 0
z_init = 0
phi_init = 0 #roll
theta_init = 0 #pitch
psi_init = 0 # yaw
x_pos_dot_init = 0
y_dot_init = 0
z_dot_init = 0
phi_dot_init = 0
theta_dot_init = 0
psi_dot_init = 0


x_pos_target = 0.09
y_target = 0.14
z_target = 0.07
phi_target = 0 #roll
theta_target = 0 #pitch
psi_target = 0.45 #yaw
x_pos_dot_target = 0
y_dot_target = 0
z_dot_target = 0
phi_dot_target = 0
theta_dot_target = 0
psi_dot_target = 0


# x˙(t) = [x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot, x_ddot, y_ddot, z_ddot, phi_ddot, ˙theta_ddot, psi_ddot]T


x_pos_ddot = (cos(phi) * sin(theta) * cos(psi) + sin(phi) * sin(psi))  * thrust / m
y_ddot = (cos(phi) * sin(theta) * cos(psi) - sin(phi) * cos(psi))  * thrust / m
z_ddot = -g + (cos(phi) * cos(theta)) * thrust / m
phi_ddot = theta_dot * psi_dot * (Iy - Iz) / (Ix) + tau_phi / Ix
theta_ddot = phi_dot * psi_dot * (Iz - Ix) / (Iy) + tau_theta / Iy
psi_ddot = theta_dot * phi_dot * (Ix - Iy) / (Iz) + tau_psi / Iz

x_dot = vertcat(x_pos_dot, y_dot, z_dot, phi_dot, theta_dot, psi_dot, x_pos_ddot, y_ddot, z_ddot, phi_ddot, theta_ddot, psi_ddot)

f = Function('f', [x, u], [x_dot], ['x', 'u'], ['x_dot'])

f

Function(f:(x[12],u[4])->(x_dot[12]) MXFunction)

In [148]:
U = MX.sym("U", Nu, Nhoriz) # Decision variables (controls)
P = MX.sym('P',Nx + Nx) #parameters (which include the initial state and the reference state)
X = MX.sym("X", Nx, Nhoriz + 1) # A vector that represents the states over the optimization problem.

In [149]:
J = 0 # Objective function
g = []  # constraints vector

#Q=diag(MX([100, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q = diagcat(100, 100, 100, 1, 1, 1, 1, 1, 1, 1, 1, 1)
#R=diag(MX([10.0, 10.0, 10.0, 10.0]))
R = diagcat(10.0, 10.0, 10.0, 10.0)



x_init = P[0:Nx]
g = X[:, 0] - P[0:Nx]  # initial condition constraints
h = 0.2
J = 0

for k in range(Nhoriz-1):  
    st_ref = P[Nx:2*Nx]
    st = X[:, k]
    cont = U[:, k]
    J += (st - st_ref).T @ Q @ (st - st_ref) + cont.T @ R @ cont
    st_next = X[:, k+1]
    k1 = f(st, cont)  
    k2 = f(st + h / 2 * k1, cont)
    k3 = f(st + h / 2 * k2, cont)
    k4 = f(st + h * k3, cont)
    st_next_RK4 = st + h/6*(k1 + 2*k2 + 2*k3 + k4)  # RK4 integration
    g = vertcat(g, st_next - st_next_RK4) # Multiple Shooting





In [150]:
OPT_variables = vertcat(
    X.reshape((-1, 1)),   # Example: 3x11 ---> 33x1 where 3=states, 11=N+1
    U.reshape((-1, 1))
)

nlp_prob = {
    'f': J,
    'x': OPT_variables,
    'g': g,
    'p': P
}

opts = {
    'ipopt': {
        'max_iter': 20,
        'print_level': 0,
        'acceptable_tol': 1e-8,
        'acceptable_obj_change_tol': 1e-6
    },
    'print_time': 0
}

solver = nlpsol('solver', 'ipopt', nlp_prob, opts)

lbx = DM.zeros((Nx*(Nhoriz+1) + Nu*Nhoriz, 1))
ubx = DM.zeros((Nx*(Nhoriz+1) + Nu*Nhoriz, 1))

lbx[0: Nx*(Nhoriz+1): Nx] = -inf     # x lower bound
lbx[1: Nx*(Nhoriz+1): Nx] = -inf     # y lower bound
lbx[2: Nx*(Nhoriz+1): Nx] = -inf     # z lower bound
lbx[3: Nx*(Nhoriz+1): Nx] = -inf     # phi lower bound
lbx[4: Nx*(Nhoriz+1): Nx] = -inf     # theta lower bound
lbx[5: Nx*(Nhoriz+1): Nx] = -inf     # psi lower bound
lbx[6: Nx*(Nhoriz+1): Nx] = -inf     # x_dot lower bound
lbx[7: Nx*(Nhoriz+1): Nx] = -inf     # y_dot lower bound
lbx[8: Nx*(Nhoriz+1): Nx] = -inf     # z_dot lower bound
lbx[9: Nx*(Nhoriz+1): Nx] = -inf     # phi_dot lower bound
lbx[10: Nx*(Nhoriz+1): Nx] = -inf     # theta_dot lower bound
lbx[11: Nx*(Nhoriz+1): Nx] = -inf     # psi_dot lower bound


ubx[0: Nx*(Nhoriz+1): Nx] = inf     # x upper bound
ubx[1: Nx*(Nhoriz+1): Nx] = inf     # y upper bound
ubx[2: Nx*(Nhoriz+1): Nx] = inf     # z upper bound
ubx[3: Nx*(Nhoriz+1): Nx] = inf     # phi upper bound
ubx[4: Nx*(Nhoriz+1): Nx] = inf     # theta upper bound
ubx[5: Nx*(Nhoriz+1): Nx] = inf     # psi upper bound
ubx[6: Nx*(Nhoriz+1): Nx] = inf     # x_dot upper bound
ubx[7: Nx*(Nhoriz+1): Nx] = inf     # y_dot upper bound
ubx[8: Nx*(Nhoriz+1): Nx] = inf     # z_dot upper bound
ubx[9: Nx*(Nhoriz+1): Nx] = inf     # phi_dot upper bound
ubx[10: Nx*(Nhoriz+1): Nx] = inf     # theta_dot upper bound
ubx[11: Nx*(Nhoriz+1): Nx] = inf     # psi_dot upper bound

lbx[Nx*(Nhoriz+1):] = -1.2                 # v lower bound for all u
ubx[Nx*(Nhoriz+1):] = 1.2                  # v upper bound for all u

# Starting index for the control variables section
start_idx_control = Nx * (Nhoriz + 1)

# Set bounds for the thrust (first control input) and tau_psi (fourth control input) across all timesteps in a single loop
for k in range(Nhoriz):
    # Index for the thrust control input at timestep k
    idx_thrust = start_idx_control + k*Nu  # First control input
    
    # Index for tau_psi at timestep k
    idx_tau_psi = start_idx_control + k*Nu + 3  # Fourth control input, hence +3
    
    # Set lower and upper bounds for thrust
    lbx[idx_thrust] = 0
    ubx[idx_thrust] = 35  # Adjusted as per your correction to 35
    
    # Set lower and upper bounds for tau_psi
    lbx[idx_tau_psi] = -0.2
    ubx[idx_tau_psi] = 0.2


# Except for
#lbx[0: Nx*(Nhoriz+1):] = 0                 # v lower bound for all u
#ubx[0: Nx*(Nhoriz+1):] = 25                 # v upper bound for all u

args = {
    'lbg': DM.zeros((Nx*Nhoriz, 1)),  # constraints lower bound
    'ubg': DM.zeros((Nx*Nhoriz, 1)),  # constraints upper bound
    'lbx': lbx,
    'ubx': ubx
}

#   'lbg': DM.zeros((Nx*(Nhoriz+1), 1)),  # constraints lower bound
#   'ubg': DM.zeros((Nx*(Nhoriz+1), 1)),  # constraints upper bound


In [151]:
t0 = 0
state_init = DM([x_pos_init, y_init, z_init, phi_init, theta_init, psi_init, x_pos_dot_init, y_dot_init, z_dot_init, phi_dot_init, theta_dot_init, psi_dot_init])        # initial state
state_target = DM([x_pos_target, y_target, z_target, phi_target, theta_target, psi_target, x_pos_dot_target, y_dot_target, z_dot_target, phi_dot_target, theta_dot_target, psi_dot_target])  # target state

# xx = DM(state_init)
t = DM(t0)

u0 = DM.zeros((Nu, Nhoriz))  # initial control
X0 = repmat(state_init, 1, Nhoriz+1)         # initial state full


def DM2Arr(dm):
    return np.array(dm.full())

mpc_iter = 0
cat_states = DM2Arr(X0)
cat_controls = DM2Arr(u0[:, 0])
times = np.array([[0]])

In [152]:
def shift_timestep(step_horizon, t0, state_init, u, f):
    f_value = f(state_init, u[:, 0])
    next_state = DM.full(state_init + (step_horizon * f_value))

    t0 = t0 + step_horizon
    u0 = horzcat(
        u[:, 1:],
        reshape(u[:, -1], -1, 1)
    )

    return t0, next_state, u0

In [153]:
main_loop = time()  # return time in sec
while (norm_2(state_init - state_target) > 1e-2) and (mpc_iter * h < 200):
    t1 = time()
    args['p'] = vertcat(
        state_init,    # current state
        state_target   # target state
    )
    # optimization variable current state
    args['x0'] = vertcat(
        reshape(X0, Nx*(Nhoriz+1), 1),
        reshape(u0, Nu*Nhoriz, 1)
    )

    sol = solver(
        x0=args['x0'],
        lbx=args['lbx'],
        ubx=args['ubx'],
        lbg=args['lbg'],
        ubg=args['ubg'],
        p=args['p']
    )

    u = reshape(sol['x'][Nx * (Nhoriz + 1):], Nu, Nhoriz)
    
    u_np = np.array(u.full())
    np.set_printoptions(precision=4, suppress=True)
    #print(u_np)
    
    print("Control Inputs (u) at this iteration:")
    print(tabulate(u_np, tablefmt="fancy_grid", showindex="always", headers=[f"Step {i}" for i in range(Nhoriz)]))

    
    X0 = reshape(sol['x'][: Nx * (Nhoriz+1)], Nx, Nhoriz+1)
    
    

    cat_states = np.dstack((
        cat_states,
        DM2Arr(X0)
    ))

    cat_controls = np.vstack((
        cat_controls,
        DM2Arr(u[:, 0])
    ))
    t = np.vstack((
        t,
        t0
    ))

    t0, state_init, u0 = shift_timestep(h, t0, state_init, u, f)

    # print(X0)
    X0 = horzcat(
        X0[:, 1:],
        reshape(X0[:, -1], -1, 1)
    )

    # xx ...
    t2 = time()
    print(mpc_iter)
    print(t2-t1)
    times = np.vstack((
        times,
        t2-t1
    ))

    mpc_iter = mpc_iter + 1

main_loop_time = time()
ss_error = norm_2(state_init - state_target)

print('\n\n')
print('Total time: ', main_loop_time - main_loop)
print('avg iteration time: ', np.array(times).mean() * 1000, 'ms')
print('final error: ', ss_error)

# simulate
# simulate(cat_states, cat_controls, times, h, Nhoriz,
#         np.array([x_init, y_init, theta_init, x_target, y_target, theta_target]), save=False)

Control Inputs (u) at this iteration:
╒════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════╕
│    │       Step 0 │       Step 1 │       Step 2 │       Step 3 │       Step 4 │       Step 5 │       Step 6 │       Step 7 │       Step 8 │   Step 9 │
╞════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════╡
│  0 │  0.324588    │  0.262513    │  0.260411    │  0.29262     │  0.284723    │  0.286025    │  0.289862    │  0.24158     │  8.55385e-05 │  2.34099 │
├────┼──────────────┼──────────────┼──────────────┼──────────────┼──────────────┼──────────────┼──────────────┼──────────────┼──────────────┼──────────┤
│  1 │ -2.34653e-05 │  4.44872e-05 │ -8.52769e-06 │ -1.37933e-05 │ -4.35093e-06 │  2.15406e-06 │  2.84218e-06 │  1.00477e-06 │ -3.02297e-07 │  0       │
├────┼──────────────┼──────────────┼────────

CasADi - 2024-02-02 17:33:25 WARNING("solver:nlp_g failed: Inf detected for output g, at (row 9, col 0).") [.../casadi/core/oracle_function.cpp:326]
